In [ ]:
spark.conf.set("fs.azure.account.key.stupramonitoreomercado.dfs.core.windows.net", "xxx")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")
dbutils.fs.ls("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/")
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "false")
#spark.conf.set("spark.sql.shuffle.partitions",3000)
#spark.conf.set("spark.default.parallelism",3000)

In [ ]:
dbutils.fs.ls("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/")

In [ ]:
dbutils.fs.head("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/RANGOS_CEDULA.json")

In [ ]:
'''
,"SASTableData+RANGOS_CEDULA":[

{"COD_NATURALEZA":"0125",
"NATURALEZA_JURIDICA":"COMPRAVENTA",
"ROL":"DE",
"PERSONA":"MARIA CRISTINA",
"TIPO_DOCUMENTO":"C",
"NUMERO_DOCUMENTO":00000,
"RANGOS_CEDULA_SEXO":"MUJER"},


'''

from pyspark.sql.functions import *
from pyspark.sql.types import *
#lines = textFile.collect()

# https://spark.apache.org/docs/latest/sql-ref-datatypes.html

schema = StructType().add(
  StructField("SASTableData+RANGOS_CEDULA", ArrayType(
      StructType([
          StructField("COD_NATURALEZA", StringType()),
          StructField("NATURALEZA_JURIDICA", StringType()),
          StructField("ROL", StringType()),
          StructField("PERSONA", StringType()),
          StructField("TIPO_DOCUMENTO", StringType()),
          StructField("NUMERO_DOCUMENTO", StringType()),
          StructField("RANGOS_CEDULA_SEXO", StringType())
      ])
   )
))


df1 = spark.read.json("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/RANGOS_CEDULA.json", schema=schema)

In [ ]:
df1.count()

In [ ]:
df1.printSchema()

In [ ]:
#df1.createOrReplaceTempView("personas")

#spark.sql("select SASTableData+RANGOS_CEDULA from personas " )

In [ ]:
#rows = df1.select(  explode( "SASTableData+RANGOS_CEDULA" )) 
#rows = df1.select( explode(  array("SASTableData+RANGOS_CEDULA").alias("row")  ) ) 

rows = df1.select( explode("SASTableData+RANGOS_CEDULA").alias("row") )
rows.count()

In [ ]:
rows.printSchema()

In [ ]:
display(rows)

In [ ]:
rows.select( "row.*").show()

In [ ]:
parsed_rows = rows.select( "row.*")

In [ ]:
parsed_rows.write.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/RANGOS_CEDULA.parquet")

In [ ]:
df2 = spark.read.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/RANGOS_CEDULA.parquet")
#display(from_json(df2.select("col"))
display(df2)

In [ ]:
df2.count()

In [ ]:
df3 = df2.dropDuplicates()
df3.count()

In [ ]:
df3.show()

In [ ]:
df3.createOrReplaceTempView("personas")

# se ignoran los Nit (N), CEDULA EXTRANJERIA O PASAPORTE (E) y los que no tienen número de documento

df3_filter  = spark.sql("""
SELECT distinct  
 regexp_replace(  regexp_replace( trim(upper(PERSONA)),  ' +', ' ') ,  ',+', ','    )   as PERSONA , 
TIPO_DOCUMENTO, NUMERO_DOCUMENTO, RANGOS_CEDULA_SEXO
FROM personas
where TIPO_DOCUMENTO <> 'N'  
and TIPO_DOCUMENTO <>   'E'
and NUMERO_DOCUMENTO IS NOT NULL
""")



df3_filter.count()
# 2894721  -- sin distinct
# 2108660  -- con distinct

In [ ]:
df3_filter.show(truncate=False) 

In [ ]:

from pyspark.sql.types import *
from pyspark.sql.functions import udf
import pyspark.sql.functions as F

# listado de funciones   https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions


def count_words(nombre):
  return len(nombre.split(" ") )

count_words_udf = udf(count_words, IntegerType() )

def count_commas(nombre):
  return len(nombre.split(",") )

count_commas_udf = udf(count_commas, IntegerType() )

#  Notas: hay algunas personas que separan apellido y nombre usando una coma 
df4_filter = df3_filter.withColumn("persona_num_parts", count_words_udf(df3_filter.PERSONA)  )
df4_filter = df4_filter.withColumn("persona_num_commas", count_commas_udf(df4_filter.PERSONA)  )
df4_filter.show(truncate=False) 

In [ ]:
df4_filter.createOrReplaceTempView("personas_filtradas")



In [ ]:
df5_filter  = spark.sql("""
SELECT persona_num_commas, count(*) 
FROM personas_filtradas
group by persona_num_commas 
order by persona_num_commas desc, count(*) desc
""")

df5_filter.show(truncate=False) 

In [ ]:
spark.sql("""
SELECT *
FROM personas_filtradas
where  persona_num_commas = 3
""").show(n= 10, truncate=False) 

In [ ]:
# se ignoran los nombres con más de 3 ","

df4_filter_comma =   spark.sql("""
SELECT *
FROM personas_filtradas
where  persona_num_commas = 2
""")
df4_filter_comma.show(n= 10, truncate=False) 

In [ ]:
split_col = F.split(df4_filter_comma['PERSONA'], ',')
df4_filter_comma = df4_filter_comma.withColumn('APELLIDOS', split_col.getItem(0))
df4_filter_comma = df4_filter_comma.withColumn('NOMBRES', split_col.getItem(1))
df4_filter_comma.show(n= 10, truncate=False) 

In [ ]:
#df4_filter_comma.select(["NOMBRES", "RANGOS_CEDULA_SEXO"]).orderBy('NOMBRES',ascending=True).show()
# df_export = df4_filter_comma.select(["NOMBRES", "RANGOS_CEDULA_SEXO"]).orderBy('NOMBRES',ascending=True)
df4_filter_comma.createOrReplaceTempView("personas_split_commma")

df_export =   spark.sql("""
SELECT distinct  trim(NOMBRES) as NOMBRES, RANGOS_CEDULA_SEXO
FROM personas_split_commma
order by NOMBRES
""")
#  distinct trim(NOMBRES), RANGOS_CEDULA_SEXO
df_export.show(n= 10, truncate=False) 


In [ ]:

df_export.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/nombres_separados_coma.json")

In [ ]:
df4_filter_sin_comma =   spark.sql("""
SELECT *
FROM personas_filtradas
where  persona_num_commas = 1
""")
df4_filter_sin_comma.show(n= 10, truncate=False) 

In [ ]:
df4_filter_sin_comma.count()

In [ ]:
'''
def remove_spaces(nombre):
  return re.sub(' +', ' ', nombre.strip())

remove_spaces_udf = udf(remove_spaces, StringType() )
#df4_filter = df3_filter.withColumn("nombre_clean", remove_punctuation_udf(df3_filter.PERSONA))
'''

## Remover doble espacio
#df4_filter = df3_filter.withColumn('nombre_clean', F.regexp_replace('PERSONA', r' +', ' '))

#remover puntuación
df5 = df4_filter_sin_comma.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA', r'[^\w\s]', ''))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DEL ', ' '))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DE LA ', ' '))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DE LAS ', ' '))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' DE ', ' '))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r' LOS ', ' '))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DEL ', ''))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DE LA ', ''))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DE LAS ', ''))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^DE ', ''))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^LOS ', ''))
df5 = df5.withColumn('PERSONA_CLEAN', F.regexp_replace('PERSONA_CLEAN', r'^LA ', ''))

#actualizar conteo de partes
df5 = df5.withColumn("persona_num_parts", count_words_udf(df5.PERSONA_CLEAN)  )

df5.show(truncate=False) 

In [ ]:
df5.createOrReplaceTempView("personas_without_comma")

spark.sql("""
SELECT * 
FROM personas_without_comma
where  persona_num_parts = 2
""").show(truncate=False) 

In [ ]:
df_export =  spark.sql("""
SELECT * 
FROM personas_without_comma
where  persona_num_parts = 2
""")

split_col = F.split(df_export['PERSONA_CLEAN'], ' ')
df_export = df_export.withColumn('APELLIDOS', split_col.getItem(0))
df_export = df_export.withColumn('NOMBRES', split_col.getItem(1))
df_export.show(n= 10, truncate=False) 



In [ ]:
df_export.createOrReplaceTempView("personas_apellido_nombre")

df_export_2 =   spark.sql("""
SELECT distinct  trim(NOMBRES) as NOMBRES, RANGOS_CEDULA_SEXO
FROM personas_apellido_nombre
order by NOMBRES
""")
#  distinct trim(NOMBRES), RANGOS_CEDULA_SEXO
#df_export.show(n= 10, truncate=False) 


df_export_2.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/nombres_apellido_nombre.json")

In [ ]:
df_export =  spark.sql("""
SELECT * 
FROM personas_without_comma
where  persona_num_parts > 2
""")

split_col = F.split(df_export['PERSONA_CLEAN'], '^(?:\S+\s){1}(\S+)'   )
#df_export = df_export.withColumn('APELLIDOS', split_col.getItem(0)       )
df_export = df_export.withColumn('NOMBRES', split_col.getItem(1)  )




df_export.show(n= 10, truncate=False) 




In [ ]:
df_export.createOrReplaceTempView("personas_apellido_nombres")

df_export_2 =   spark.sql("""
SELECT distinct  trim(NOMBRES) as NOMBRES, RANGOS_CEDULA_SEXO
FROM personas_apellido_nombres
order by NOMBRES
""")
#  distinct trim(NOMBRES), RANGOS_CEDULA_SEXO
#df_export.show(n= 10, truncate=False) 


df_export_2.coalesce(1).write.json("abfss://sandbox@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/nombres_apellido_nombres.json")

In [ ]:
spark.sql("""
SELECT persona_num_parts, count(*) 
FROM personas_without_comma
group by persona_num_parts 
order by persona_num_parts asc
""").show(truncate=False) 

In [ ]:
spark.sql("""
SELECT *
FROM personas_without_comma
where  persona_num_parts = 1 
""").show(n= 50, truncate=False) 

In [ ]:
dfwc  = spark.sql("""
SELECT persona_num_parts, count(*) 
FROM personas_without_comma
group by persona_num_parts 
order by persona_num_parts asc, count(*) desc
""")

dfwc.show(truncate=False) 

In [ ]:
df4_filter.createOrReplaceTempView("personas_filtradas")

df5_filter  = spark.sql("""
SELECT num_parts_nombre, count(*) 
FROM personas_filtradas
group by num_parts_nombre order by num_parts_nombre desc, count(*) desc
""")

df5_filter.show(truncate=False) 

In [ ]:
df5_filter  = spark.sql("""
SELECT * 
FROM personas_filtradas
where num_parts_nombre > 20
""")

#   tipo_documento = 'E'
df5_filter.show(truncate=False) 

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
pd.__version__



In [ ]:
df = df2.select("*").toPandas()
profile = ProfileReport(df, title="Pandas Profiling Report")
displayHTML(profile.to_html())


In [ ]:
dbutils.fs.put("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/RANGOS_CEDULA.parquet_profile.html", profile.to_html() )

In [ ]:
'''
Requirements
Apache Spark>= 2.4.0
Python>=3.6

No funciona: parece que hay  incompatiblidad con versiones recientes de pandas
'''

from pyspark.sql import SparkSession
from optimus import Optimus

spark = SparkSession.builder.appName('optimus').getOrCreate()
op= Optimus(spark)

odf = op.load.parquet("abfss://standarized@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/RANGOS_CEDULA.parquet")

In [ ]:
df_sin_duplicados = df.drop_duplicates()

In [ ]:
df_sin_duplicados.shape

In [ ]:
profile = ProfileReport(df_sin_duplicados, title="Pandas Profiling Report")
dbutils.fs.put("abfss://raw@stupramonitoreomercado.dfs.core.windows.net/OTROS/SNR/JSON/RANGOS_CEDULA_v3.parquet_profile_sin_duplicados.html", profile.to_html() )